# Assignment 2: Retrieval-Augmented Question Answering with LLMs

In this assignment, we will use a large language model (LLM) in a retrieval-augmented setup to answer questions from the Natural Questions dataset. We will evaluate the performance of the LLM using different prompting strategies and compare the results. The steps involved are as follows:

1. **Evaluate an LLM on Natural Questions without context.**
2. **Evaluate an LLM on Natural Questions with provided context.**
3. **Set up a retriever to find relevant passages.**
4. **Evaluate the LLM using retrieved context instead of provided context.**


In [2]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

nq_data = pd.read_csv('nq_simplified.val.tsv', sep='\t', header=None, names=['question', 'answer', 'gold_context'], quoting=3)

nq_data.head()


,question,answer,gold_context
0,what purpose did seasonal monsoon winds have o...,enabled European empire expansion into the Ame...,The westerlies (blue arrows) and trade winds (...
1,who got the first nobel prize in physics,"Wilhelm Conrad Röntgen, of Germany",The award is presented in Stockholm at an annu...
2,when is the next deadpool movie being released,"May 18, 2018","Though the original creative team of Reynolds,..."
3,where did the idea of fortnite come from,as a cross between Minecraft and Left 4 Dead,"Fortnite is set in contemporary Earth, where t..."
4,which mode is used for short wave broadcast se...,MFSK Olivia,"All one needs is a pair of transceivers, each ..."


In [3]:
def rouge1(gold, predicted):
    assert(len(gold) == len(predicted))
    n_p = 0
    n_g = 0
    n_c = 0
    for g, p in zip(gold, predicted):
        g = set(cleanup(g).strip().split())
        p = set(cleanup(p).strip().split())
        n_g += len(g)
        n_p += len(p)
        n_c += len(p.intersection(g))
    pr = n_c / n_p
    re = n_c / n_g
    if pr > 0 and re > 0:
        f1 = 2*pr*re/(pr + re)
    else:
        f1 = 0.0
    return pr, re, f1

def cleanup(text):
    text = text.replace(',', ' ')
    text = text.replace('.', ' ')
    return text

## Step 1: Evaluating an LLM on Natural Questions

First, we will find an LLM on huginface hub that is small and fast to fitt on our system. 

In [12]:
# Load the model and tokenizer
# "mistralai/Mixtral-8x7B-Instruct-v0.1" "microsoft/Phi-3-mini-4k-instruct" "mtgv/MobileLLaMA-1.4B-Base" 
model_name = "microsoft/Phi-3-mini-4k-instruct"  # 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
model.eval()
# model.to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

Lets create a simple pipline for QA and use a small subset of the dataset to get the predicted answers and evaluate the model using ROUGE-1 scores.

In [24]:
# Create a pipeline for question answering
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define a function to get answers from the model
def get_answers(questions):
    answers = []
    for question in questions:
        response = qa_pipeline(question, max_new_tokens=64)
        answers.append(response[0]['generated_text'])
    return answers

# Use a small subset of the dataset for testing
subset = nq_data.sample(1)
questions = subset['question'].tolist()
gold_answers = subset['answer'].tolist()

predicted_answers = get_answers(questions)

# Evaluate the model
pr, re, f1 = rouge1(gold_answers, predicted_answers)
print(f"ROUGE-1 Precision: {pr}, Recall: {re}, F1 Score: {f1}")

ROUGE-1 Precision: 0.0, Recall: 0.0, F1 Score: 0.0


## Step 2: An Idealized Retrieval-Augmented LLM

Now, we will include the `gold_context` in our prompts and evaluate the model again. This will help us understand the upper limits of the system.


In [25]:
# Define a function to get answers from the model with context
def get_answers_with_context(questions, contexts):
    answers = []
    for question, context in zip(questions, contexts):
        prompt = f"Question: {question}\nContext: {context}\nAnswer:"
        response = qa_pipeline(prompt, max_new_tokens=64)
        answers.append(response[0]['generated_text'])
    return answers

contexts = subset['gold_context'].tolist()

predicted_answers_with_context = get_answers_with_context(questions, contexts)

pr_context, re_context, f1_context = rouge1(gold_answers, predicted_answers_with_context)
print(f"With Context - ROUGE-1 Precision: {pr_context}, Recall: {re_context}, F1 Score: {f1_context}")


With Context - ROUGE-1 Precision: 0.03225806451612903, Recall: 1.0, F1 Score: 0.0625


## Step 3: Setting Up the Retriever

In this step, we will set up a retriever to find relevant passages. We will use the SentenceTransformers model to create embeddings for the passages and set up a FAISS index for efficient retrieval.


In [32]:
# !pip install faiss-cpu #windows

     --------------------------------------- 14.5/14.5 MB 14.9 MB/s eta 0:00:00


You should consider upgrading via the 'P:\wasp_deep_learning_for_natural_language_processing_2024\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

with open('passages.txt', 'r', encoding='utf-8') as file:
    passages = file.readlines()

embedded_passages = retriever_model.encode(passages, convert_to_tensor=True)

index = faiss.IndexFlatL2(embedded_passages.shape[1])
index.add(embedded_passages.cpu())


In [37]:
def retrieve_best_passage(question):
    question_embedding = retriever_model.encode([question], convert_to_tensor=True)
    _, ix = index.search(question_embedding.cpu(), 1)
    return passages[ix[0][0]]

question = "Where did the first African American air force unit train?"
best_passage = retrieve_best_passage(question)
print(f"Best Passage: {best_passage}")


Best Passage: Black Americans were not permitted to fly for the U.S. armed services prior to 1940. The Air Corps at that time, which had never had a single black member, was part of an army that possessed exactly two black Regular line officers at the beginning of World War II: Brigadier Generals Benjamin O. Davis, Sr. and Benjamin O. Davis, Jr. The first Civilian Pilot Training Program (CPTP) students completed their instruction in May 1940. The creation of an all-black pursuit squadron resulted from pressure by civil rights organizations and the black press who pushed for the establishment of a unit at Tuskegee, an



## Step 4: Putting the Pieces Together

Finally, we will retrieve passages for each question and evaluate the model using these passages instead of the provided `gold_context`.

In [38]:
def get_answers_with_retrieved_context(questions):
    answers = []
    for question in questions:
        context = retrieve_best_passage(question)
        prompt = f"Question: {question}\nContext: {context}\nAnswer:"
        response = qa_pipeline(prompt, max_new_tokens=64)
        answers.append(response[0]['generated_text'])
    return answers

predicted_answers_retrieved_context = get_answers_with_retrieved_context(questions)

pr_retrieved, re_retrieved, f1_retrieved = rouge1(gold_answers, predicted_answers_retrieved_context)
print(f"With Retrieved Context - ROUGE-1 Precision: {pr_retrieved}, Recall: {re_retrieved}, F1 Score: {f1_retrieved}")


With Retrieved Context - ROUGE-1 Precision: 0.02127659574468085, Recall: 1.0, F1 Score: 0.04166666666666667


## Summary

In this notebook, we evaluated the performance of an LLM on the Natural Questions dataset using different prompting strategies. We observed the following:

1. **Without Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

2. **With Provided Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

3. **With Retrieved Context**:
   - ROUGE-1 Precision: *result*
   - Recall: *result*
   - F1 Score: *result*

Including context, whether provided or retrieved, ...
